In [0]:
# What this script does: Creates gantt charts which match data in the database with project requirements.

# 1) Retrieves list of Tables & sizes from the (cip_uk5) database
# 2) Reads in list of Projects & their data requirements
# 3) Separates tables into monthly and daily timeframes
# 4) Creates (long) gantt charts for of the two inputs (i.Tables; ii.Projects), for each timeframe
# 5) Combine the gantt chart from each input into an overall gantt, for each timeframe

# 6) This overall gantt includes all tables as columns, and all dates as the index.
# >> The values in each cell represent what should be done to the table in the database:
# a) 'Null' = if not in Tables, not needed in Projects
# b) 'Drop' = if in Tables, not needed in Projects
# c) 'Add' = if not in Tables, needed in Projects (including the project names that require this data)
# d) 'Keep' = if in Tables, needed in Projects (including the project names that require this data)

# 7) Adjust output of overall gantts to take into account Jira tickets
# 8) Dataframe of how much space is taken up by the tables associated with each project

# 9) Creates dataframe, from Tables input, of how much space each schema takes up in the database
# 10) Export as a csv the two size tables, the two overall gantts, and the one space dataframe


# NB: in order to preserve IP, I have replaced any database information with '***'

In [0]:
# Basic imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Displays all cell's output, not just last output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# For pd.read_excel
import xlrd

# Setting plotting styles
plt.style.use('fivethirtyeight')
sns.set_style('white')

# From the generic function to connect to db
import os
import psycopg2 # Package for communication with postgres
from datetime import datetime, date, timedelta, time
import time
import random as rd

In [0]:
# Define function to retrieve table from database - GENERIC

def getDataFromPostGre(stmt, pg_database='***', pg_user='***', \
                   pg_password='***', pg_host='localhost', pg_port='***'):
    '''
    Function that gets data from a postgres database
        stmt = SQL statement to extract table from database
    Returns:
        The values fetched within a dataframe if ok, False if not
    '''
    # DATABASE CONNECTION FOR READ ONLY
    try:
        conn = psycopg2.connect(database=pg_database, \
                                user=pg_user, \
                                password=pg_password, \
                                host=pg_host, \
                                port=pg_port)
        print("Connexion to cip_uk5 successful")
    except:
        print('#######################################################')
        print ("Unable to open database ", pg_database)
        print ('#######################################################')
        return False
    # Database connection
    cur = conn.cursor()
    # Query construction
    query = stmt
    # Query execution
    print(query), ' to be executed'
    cur.execute(query)
    # Commit the transaction
    conn.commit()
    # Get the data in a table
    data = cur.fetchall()
    # Extract col names
    columns = [desc[0] for desc in cur.description]
    # Read the data into a Pandas DF
    table_data = pd.DataFrame(data, columns = columns)
    print(len(table_data.index), "Records fetched successfully")
    # Close connection
    conn.close()
    return table_data

In [0]:
# 1) Retrieves list of Tables & sizes from the (cip_uk5) database

# Retrieve data from database using fn defined above
stmt = '***'
all_tables_df = getDataFromPostGre(stmt=stmt) 

# Select only cip_partition schema
tables_df = all_tables_df[all_tables_df.schemaname == '***'].drop('***', axis=1).copy()

# Categorise as monthly or daily - based on the suffix of YYYYmm or YYYYmmdd
tables_df['timeframe'] = tables_df.relname.apply(lambda x: 'monthly' if len(x.split('_')[-1])==6\
                                                            else 'daily')

# Extract & parse date
tables_df['date'] = tables_df.relname.apply(
    lambda x: pd.to_datetime(x.split('_')[-1], format='%Y%m') if len(x.split('_')[-1]) == 6 \
    else pd.to_datetime(x.split('_')[-1], format='%Y%m%d'))

# Extract pure table name
tables_df['table'] = tables_df.relname.apply(lambda x: '_'.join(x.split('_')[:-1]))

# Extract Size measurement & unit
tables_df['size_num'] = tables_df.Size.apply(
    lambda x: int(x.split(' ')[0]))
tables_df['size_unit'] = tables_df.Size.apply(
    lambda x: x.split(' ')[-1])

# Function to create standardised size in MBs
def into_MBs(row):
    if row['size_unit']=='MB':
      return row['size_num']
    elif row['size_unit']=='GB':
      return row['size_num']*1000
    elif row['size_unit']=='kB':
      return row['size_num']/1000
    elif row['size_unit']=='bytes':
      return row['size_num']/10**6

tables_df['size_MBs'] = tables_df.apply(lambda row: into_MBs(row), axis=1)

# Set Index & drop uneccessary columns
tables_df = tables_df.set_index(['table', 'timeframe', 'date']).drop(['Size', 'size_num', 'size_unit'], axis=1)

In [0]:
# 2) Reads in list of Projects & their data requirements

# Read in Projects data
HUB_projects_df = pd.read_excel('***', 
                                sheet_name='***', 
                                header=3)
HUB_projects_df.head(2)

# Select only relevant tables, and if the table still needed
HUB_projects_df = HUB_projects_df[(HUB_projects_df.Database == '***') & 
                                 (HUB_projects_df['Data Removed'] != 'Y')]

# Select only relevant columns, and rename
projects_csv_df = HUB_projects_df[['Project Name', 'Data Table', 'Data Start', 'Data End']].copy()
projects_csv_df.columns = ['project', 'table', 'start', 'end']

# Lowercase table name as standard
projects_csv_df['table'] = projects_csv_df.table.apply(lambda x: x.lower())

# Date columns into datetime format
projects_csv_df['start'] = pd.to_datetime(projects_csv_df['start'], infer_datetime_format=True)
projects_csv_df['end'] = pd.to_datetime(projects_csv_df['end'], infer_datetime_format=True)

# Set index to format nicely
projects_csv_df = projects_csv_df.set_index(['project', 'table'])

In [0]:
# Checking Tables & Projects dfs

tables_df.head(2)
projects_csv_df.head(2)

In [0]:
# 3) Separates tables into monthly and daily timeframes
# 4)a) Creates (long) gantt charts for Projects, for each timeframe

# Select just monthly tables, then pivot to produce a Gantt
monthly_tables_gantt = tables_df.loc[(slice(None), 'monthly', slice(None)), :]\
                        .reset_index()\
                        .pivot(index='date', columns='table', values='size_MBs')
# Displaying monthly dates as months not first days
monthly_tables_gantt.index = monthly_tables_gantt.index.to_period('M')

# Select just daily tables, then pivot to produce a Gantt
daily_tables_gantt = tables_df.loc[(slice(None), 'daily', slice(None)), :]\
                        .reset_index()\
                        .pivot(index='date', columns='table', values='size_MBs')

# Complete list of tables which should be represented in database
# Doesn't need to be alphabetical because sorted later
fulllist_monthly_tables = ['***']
fulllist_daily_tables = ['***']

# Add any columns which should be represented in database, but contain no data (so don't appear)
for col in fulllist_monthly_tables:
    if col not in monthly_tables_gantt.columns:
        monthly_tables_gantt[col] = np.nan
for col in fulllist_daily_tables:
    if col not in daily_tables_gantt.columns:
        daily_tables_gantt[col] = np.nan

# Order columns alphabetically
monthly_tables_gantt = monthly_tables_gantt[sorted(monthly_tables_gantt.columns)]
daily_tables_gantt = daily_tables_gantt[sorted(daily_tables_gantt.columns)]

# Remove any sizes which are smaller than 0.1MB as these tables do not really contain data,
# but may have leftover tiny size as a legacy of previous data stored in them
monthly_tables_gantt = monthly_tables_gantt.where((monthly_tables_gantt > 0.1) \
                                                  & (monthly_tables_gantt is not None))
daily_tables_gantt = daily_tables_gantt.where((daily_tables_gantt > 0.1) \
                                                  & (daily_tables_gantt is not None))

# Checking results
monthly_tables_gantt.head(2)
daily_tables_gantt.head(2)

In [0]:
# 4)b) Creates (long) gantt charts for Projects, for each timeframe

# If a table appears in the projects input, but is not present in the database or in the above list of tables:
# it is assumed to be a daily table and added to the complete list of daily tables.
# This means no tables mentioned in the projects input are missed.
# >> Dictionary which contains lists of all tables in each of the Tables gantts
fulllist_tables_dict = {
    'monthly': monthly_tables_gantt.columns.tolist(),
    'daily': daily_tables_gantt.columns.tolist()
}
for table in projects_csv_df.reset_index()['table'].unique():
    if table not in set(fulllist_tables_dict['daily']).union(set(fulllist_tables_dict['monthly'])):
        fulllist_tables_dict['daily'].append(table)

# Function which takes Projects df, selects the appropriate timeframe (monthly or daily), 
# and creates a project_gantt for that timeframe
def projects_gantt(timeframe ='monthly', freq='M'):
    
    # Create list of unique table names in appropriate timeframe (e.g. monthly or daily)
    timeframe_tables_list = fulllist_tables_dict[timeframe]
    
    # Extracting tables which are only for the appropriate timeframe, then set index again
    timeframe_projects = projects_csv_df.reset_index().loc[
                                projects_csv_df.reset_index().table.isin(timeframe_tables_list)]\
                                .set_index(['project', 'table'])
    
    # Creating empty dataframe with index covering entire date range, and columns as unique table names 
    timeframe_projects_gantt = pd.DataFrame(
                        index=pd.date_range(start=timeframe_projects.start.min(), 
                                            end=timeframe_projects.end.max(), freq=freq),
                        columns=timeframe_tables_list)
    
    # For each row containing project, table, start date, end date
    for row in timeframe_projects.iterrows():
        start_dt = row[1][0]
        end_dt = row[1][1]
        project = row[0][0]
        table = row[0][1]
        # Create date range between start and end for project, then loop over
        for date in pd.date_range(start=start_dt, end=end_dt, freq=freq):
            # Want to fill in the project name into the cell
            # If the cell already contains a project, want to add project name on top (not replace)
            if pd.notnull(timeframe_projects_gantt.loc[date, table]):
                timeframe_projects_gantt.loc[date, table] = timeframe_projects_gantt\
                                                                .loc[date, table]+'--'+project
            else:
                timeframe_projects_gantt.loc[date, table] = project
                
    return timeframe_projects_gantt

# Calling function for Monthly & Daily
# 'M' not 'MS' as otherwise adds an extra month on end (Jan 2019)
monthly_projects_gantt = projects_gantt(timeframe ='monthly', freq='M')
daily_projects_gantt = projects_gantt(timeframe ='daily', freq='D')

# Displaying monthly dates as months not last days
monthly_projects_gantt.index = monthly_projects_gantt.index.to_period('M')

# Checking results
monthly_projects_gantt.head(2)
daily_projects_gantt.head(2)

In [0]:
# 5)a) DAILY: Combine the gantt chart from each input into an overall gantt
# 6)a) This overall gantt includes all tables as columns, and all dates as the index.

# Get union set of all daily dates, then use this to create daterange
all_dates_daily = sorted(set(daily_projects_gantt.index).union(set(daily_tables_gantt.index)))
full_daterange_daily = pd.date_range(start=min(all_dates_daily), end=max(all_dates_daily), freq='D')

# Get union set of all daily columns
all_columns_daily = sorted(set(daily_projects_gantt.columns).union(set(daily_tables_gantt.columns)))

# Ensure both tables_gantt and projects_gantt have entire set of columns,
# by adding any columns (with all values as NaNs) to each gantt if they aren't already present
for col in all_columns_daily:
    if col not in daily_projects_gantt.columns:
        daily_projects_gantt[col] = np.nan
    if col not in daily_tables_gantt.columns:
        daily_tables_gantt[col] = np.nan

# Order columns alphabetically for both gantts 
daily_projects_gantt = daily_projects_gantt[sorted(daily_projects_gantt.columns)]
daily_tables_gantt = daily_tables_gantt[sorted(daily_tables_gantt.columns)]

# Reindex both gantts to include full daterange. If date currently not there, fills NaN for all columns
daily_projects_gantt = daily_projects_gantt.reindex(index=full_daterange_daily)
daily_tables_gantt = daily_tables_gantt.reindex(index=full_daterange_daily)

# Create frame of overall_gantt, with index as full daterange and columns as all columns
daily_overall_gantt = pd.DataFrame(index=full_daterange_daily, columns=all_columns_daily)

# For loop deploys actions for each column
# 'Null' = if not in Tables, not needed in Projects
# 'Drop' = if in Tables, not needed in Projects
# 'Add' = if not in Tables, needed in Projects
# 'Keep' = if in Tables, needed in Projects
for col in daily_overall_gantt.columns:
    # For loop over entire daterange
    for date in daily_overall_gantt.index:
        # Not in Tables
        if pd.isnull(daily_tables_gantt.loc[date,col]):
            # Whether in Projects
            if pd.isnull(daily_projects_gantt.loc[date,col]):
                daily_overall_gantt.loc[date,col] = 'Null'
            elif pd.notnull(daily_projects_gantt.loc[date,col]):
                daily_overall_gantt.loc[date,col] = 'Add: '+daily_projects_gantt.loc[date,col]
            else:
                daily_overall_gantt.loc[date,col] = 'Unsure1'
        # Is in Tables
        elif pd.notnull(daily_tables_gantt.loc[date,col]):
            # Whether in Projects
            if pd.isnull(daily_projects_gantt.loc[date,col]):
                        daily_overall_gantt.loc[date,col] = 'Drop'
            elif pd.notnull(daily_projects_gantt.loc[date,col]):
                daily_overall_gantt.loc[date,col] = 'Keep: '+daily_projects_gantt.loc[date,col]
            else:
                daily_overall_gantt.loc[date,col] = 'Unsure2'
        # If something has gone wrong..
        else:
            daily_overall_gantt.loc[date,col] = 'Unsure3'

# Check results
daily_overall_gantt.head(2)

In [0]:
# 5)a) MONTHLY: Combine the gantt chart from each input into an overall gantt
# 6)a) This overall gantt includes all tables as columns, and all dates as the index.

# Get union set of all monthly dates, then use this to create daterange
# NB. more complicated because monthly dates stored as 'Periods', hence the extra syntax
all_dates_monthly = sorted(
                        set(monthly_projects_gantt.index)\
                        .union(set(monthly_tables_gantt.index))
                        )
full_daterange_monthly = pd.date_range(
                            start=pd.to_datetime(str(min(all_dates_monthly))),
                            end=pd.to_datetime(str(max(all_dates_monthly))), 
                            freq='MS')\
                            .to_period('M').tolist()

# Get union set of all monthly columns
all_columns_monthly = sorted(set(monthly_projects_gantt.columns).union(set(monthly_tables_gantt.columns)))

# Ensure both tables_gantt and projects_gantt have entire set of columns,
# by adding any columns (with all values as NaNs) to each gantt if they aren't already present
for col in all_columns_monthly:
    if col not in monthly_projects_gantt.columns:
        monthly_projects_gantt[col] = np.nan
    if col not in monthly_tables_gantt.columns:
        monthly_tables_gantt[col] = np.nan

# Order columns alphabetically for both gantts 
monthly_projects_gantt = monthly_projects_gantt[sorted(monthly_projects_gantt.columns)]
monthly_tables_gantt = monthly_tables_gantt[sorted(monthly_tables_gantt.columns)]

# Reindex both gantts to include full daterange. If date currently not there, fills NaN for all columns
# NB. more complicated because monthly dates stored as 'Periods', hence the extra syntax
monthly_projects_gantt.index = monthly_projects_gantt.index.tolist()
monthly_tables_gantt.index = monthly_tables_gantt.index.tolist()

monthly_projects_gantt = monthly_projects_gantt.reindex(index=full_daterange_monthly)
monthly_tables_gantt = monthly_tables_gantt.reindex(index=full_daterange_monthly)

# Create frame of overall_gantt, with index as full daterange and columns as all columns
monthly_overall_gantt = pd.DataFrame(index=full_daterange_monthly, columns=all_columns_monthly)

# For loop deploys actions for each column
# 'Null' = if not in Tables, not needed in Projects
# 'Drop' = if in Tables, not needed in Projects
# 'Add' = if not in Tables, needed in Projects
# 'Keep' = if in Tables, needed in Projects
for col in monthly_overall_gantt.columns:
    # For loop over entire daterange 
    for date in monthly_overall_gantt.index:
        # Not in Tables
        if pd.isnull(monthly_tables_gantt.loc[date,col]):
            # Whether in Projects
            if pd.isnull(monthly_projects_gantt.loc[date,col]):
                monthly_overall_gantt.loc[date,col] = 'Null'
            elif pd.notnull(monthly_projects_gantt.loc[date,col]):
                monthly_overall_gantt.loc[date,col] = 'Add: '+monthly_projects_gantt.loc[date,col]
            else:
                monthly_overall_gantt.loc[date,col] = 'Unsure1'
        # Is in Tables
        elif pd.notnull(monthly_tables_gantt.loc[date,col]):
            # Whether in Projects
            if pd.isnull(monthly_projects_gantt.loc[date,col]):
                monthly_overall_gantt.loc[date,col] = 'Drop'
            elif pd.notnull(monthly_projects_gantt.loc[date,col]):
                monthly_overall_gantt.loc[date,col] = 'Keep: '+monthly_projects_gantt.loc[date,col]
            else:
                monthly_overall_gantt.loc[date,col] = 'Unsure2'
        # If something has gone wrong..
        else:
            monthly_overall_gantt.loc[date,col] = 'Unsure3'

# Check results
monthly_overall_gantt.head(2)

In [0]:
# 7) Adjust output of overall gantts to take into account Jira tickets, for both timeframes

# Read in Projects data
old_monthly_gantt = pd.read_excel('***', 
                                sheet_name='***', 
                                header=2, index_col=0)
old_daily_gantt = pd.read_excel('***', 
                                sheet_name='***', 
                                header=2, index_col=0)
old_monthly_gantt.head(2)
old_daily_gantt.head(2)

# Make sure old gantt chart contains only columns from new overall gantt
old_monthly_gantt = old_monthly_gantt[monthly_overall_gantt.columns]
old_daily_gantt = old_daily_gantt[daily_overall_gantt.columns]

# If 'jira' in old gantt, fill that cell in the new gantt with the jira link
for col in old_monthly_gantt.columns:
    for date in old_monthly_gantt.index:
        if 'jira.' in old_monthly_gantt.loc[date,col]:
            if 'Add:' in monthly_overall_gantt.loc[date,col]:
                monthly_overall_gantt.loc[date,col] = old_monthly_gantt.loc[date,col]
for col in old_daily_gantt.columns:
    for date in old_daily_gantt.index:
        if 'jira.' in old_daily_gantt.loc[date,col]:
            if 'Add:' in daily_overall_gantt.loc[date,col]:
                daily_overall_gantt.loc[date,col] = old_daily_gantt.loc[date,col]

In [0]:
# 8) Dataframe of how much space is taken up by the tables associated with each project

# Creates skeleton df
projects_sizes = pd.DataFrame(
    index=sorted(projects_csv_df.reset_index().project.unique()), 
    columns=['size_MBs'], 
    data=0)

# MONTHLY: Add the relevant datatable sizes to the projects_sizes df
# For all dates and tablenames in tables_gantt
for date in monthly_tables_gantt.index:
    for column in monthly_tables_gantt.columns:
        # Only if it's a 'Keep' will the data be in the database AND it is needed by a project
        if 'Keep:' in monthly_overall_gantt.loc[date,column]:
            # This covers date-tablename combinations which are required for multiple projects.
            # Add the size to each project
            split_projects = monthly_projects_gantt.loc[date,column].split('--')
            for project in split_projects:
                # Add datatable size to the appropriate project in the projects_sizes df 
                projects_sizes.loc[project, 'size_MBs'] += monthly_tables_gantt.loc[date,column]

# MONTHLY: Add the relevant datatable sizes to the projects_sizes df
# For all dates and tablenames in tables_gantt
for date in daily_tables_gantt.index:
    for column in daily_tables_gantt.columns:
        # Only if it's a 'Keep' will the data be in the database AND it is needed by a project
        if 'Keep:' in daily_overall_gantt.loc[date,column]:
            # This covers date-tablename combinations which are required for multiple projects.
            # Add the size to each project
            split_projects = daily_projects_gantt.loc[date,column].split('--')
            for project in split_projects:
                # Add datatable size to the appropriate project in the projects_sizes df 
                projects_sizes.loc[project, 'size_MBs'] += round(daily_tables_gantt.loc[date,column])

# Check I'm not being dumb
projects_sizes.head()

In [0]:
# 9) Creates dataframe, from Tables input, of how much space each schema takes up in the database

# Check imported tables df
all_tables_df.head(2)

# Create copy of base tables df
other_tables_df = all_tables_df.copy()

# Extract Size measurement & unit
other_tables_df['size_num'] = other_tables_df['Size'].apply(
    lambda x: int(x.split(' ')[0]) if x is not None
    else np.nan)
other_tables_df['size_unit'] = other_tables_df.Size.apply(
    lambda x: x.split(' ')[-1] if x is not None
    else np.nan)

# Used pre-defined function (near top of script) to create standardised size in MBs
other_tables_df['size_MBs'] = other_tables_df.apply(lambda row: into_MBs(row), axis=1)

# Groupby Schema name, summing sizes in MB
table_size_df = pd.DataFrame(other_tables_df.groupby('***').size_MBs.sum()).reset_index()

# playpen tables are stored under 'playpen'('***'), whereas the rest are stored under main ('***')
table_size_df['partition'] = table_size_df.apply(lambda row: 'playpen' if row['***'][:7] == '***' \
                                                          else 'main', axis=1)
size_main = '***'
size_playpen = '***'

# Create column to display percentage of storage taken up by this schema
table_size_df['perc_storage'] = table_size_df.apply(
                                    lambda row: row['size_MBs']/(size_main) if row['partition'] == 'main' \
                                    else row['size_MBs']/(size_playpen), axis=1)

# Check output
table_size_df.head(2)

# Create vars which sum the size of tables for each partition
used_size_playpen = table_size_df[table_size_df.partition == 'playpen'].size_MBs.sum()
used_size_main = table_size_df[table_size_df.partition == 'main'].size_MBs.sum()

# Created another df of 'unused' space
unused_df = pd.DataFrame(
    data={
        '***':['unused']*2,
        'size_MBs':[size_main-used_size_main, size_playpen-used_size_playpen],
        'partition':['main', 'playpen'],
        'perc_storage':[(size_main - used_size_main) / size_main, 
                        (size_playpen - used_size_playpen) / size_playpen]
    } ,
    columns=table_size_df.columns)

# Append the two tablesize dfs into one final output
final_tablesize_df = unused_df.append(table_size_df, ignore_index=True)
final_tablesize_df.head(4)

In [0]:
# 10) Export as a csv the two size tables, the two overall gantts, and the one space dataframe

projects_sizes.to_csv('project_sizes.csv', index=True)

daily_overall_gantt.to_csv('daily_overall_gantt.csv', index=True)
monthly_overall_gantt.to_csv('monthly_overall_gantt.csv', index=True)

final_tablesize_df.to_csv('schema_sizes.csv', index=False)